<a href="https://colab.research.google.com/github/Knowsoeun/AI_Study/blob/main/(CNN)Cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os

batch_size = 32
num_classes = 10
epochs = 100
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))  # layer1 : 32 32x32 Feature Maps이 출력된다.
"""
padding='same' : 입력과 출력이 가로, 세로 크기가 같도록 padding을 적절하게 넣어주라는 의미.
same이므로 그대로 두라는 의미.
"""
model.add(Activation('relu')) # layer2 : 활성화 함수 relu
model.add(Conv2D(32, (3, 3))) # layer3 : 32 32 30x30 Feature Maps이 출력된다.
model.add(Activation('relu')) # layer4 : 활성화 함수 relu
model.add(MaxPooling2D(pool_size=(2, 2))) # layer5 : 가로, 세로 길이가 반씩 줄어 32 15x15 Feature Maps이 출력된다. 
model.add(Dropout(0.25)) # layer6 : 학습을 용이하게 하기 위해 넣는 층

model.add(Conv2D(64, (3, 3), padding='same')) # layer8 : 64 15x15 Feature Maps이 출력된다.
model.add(Activation('relu')) # layer9 : 활성화 함수 relu
model.add(Conv2D(64, (3, 3))) # layer10 : 64 13x13 Feature Maps이 출력된다.
model.add(Activation('relu')) # layer11 : 활성화 함수 relu
model.add(MaxPooling2D(pool_size=(2, 2))) # layer12 : 가로, 세로 길이가 반씩 줄어 64 6x6 Feature Maps이 출력된다.
model.add(Dropout(0.25)) # layer13 : 학습을 용이하게 하기 위해 넣는 층

model.add(Flatten()) # layer14 : 합성곱층3차원 -> 완전연결층1차원이므로 Flatten(3차원을 1차원으로 구조 변경)을 쓴다.
model.add(Dense(512)) # layer15 : 완전연결층. (출력 512개. 활성화 함수 relu)
model.add(Activation('relu'))
model.add(Dropout(0.5)) # layer17 : 학습을 용이하게 하기 위해 넣는 층
model.add(Dense(num_classes)) # layer18 : 완전연결층. (클래스의 개수만큼 출력)
model.add(Activation('softmax')) # layer19 : 활성화 함수 softmax

# summarize model.
model.summary()

In [ ]:

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
